In [1]:
from matplotlib import colors
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [39]:
def resize_image(vegi):
    height, width, c = vegi.shape
    new_height = 100
    ratio = new_height / height 
    new_width = int(width * ratio)
    dimensions = (new_width, new_height)
    print(dimensions)
    resized_img = cv2.resize(vegi, dimensions, interpolation=cv2.INTER_LINEAR)
    print("New shape:      ", resized_img.shape)

    return resized_img

# current values for borders they roughly fit to the real hsv values. I test it for brown onion, carrot, brown potato with
# white background, respectively
def mask_image(img_hsv, img_rgb, border_low=(8, 50, 75), border_high=(15, 255, 160)):
    #15, 23, 135
    mask = cv2.inRange(img_hsv, border_low, border_high)
    result = cv2.bitwise_and(img_rgb, img_rgb, mask=mask)

    #plt.subplot(1, 3, 1)
    #plt.imshow(img_rgb)
    #plt.subplot(1, 3, 2)
    #plt.imshow(mask, cmap='gray')
    #plt.subplot(1, 3, 3)
    #plt.imshow(result)
    #plt.show()
    return mask, result
    
    
def resize_to_square(vegi):
    img_size = 256
    height, width = vegi.shape[:2]
    a1 = width / height
    a2 = height / width

    if (a1 > a2):
        r_img = cv2.resize(vegi, (round(img_size * a1), img_size), interpolation = cv2.INTER_AREA)
        margin = int(r_img.shape[1]/6)
        crop_img = r_img[0:img_size, margin:(margin+img_size)]

    elif(a1 < a2):
        # if height greater than width
        r_img = cv2.resize(vegi, (img_size, round(img_size * a2)), interpolation = cv2.INTER_AREA)
        margin = int(r_img.shape[0]/6)
        crop_img = r_img[margin:(margin+img_size), 0:img_size]

    elif(a1 == a2):
        # if height and width are equal
        r_img = cv2.resize(vegi, (img_size, round(img_size * a2)), interpolation = cv2.INTER_AREA)
        crop_img = r_img[0:img_size, 0:img_size]

    if(crop_img.shape[0] != img_size or crop_img.shape[1] != img_size):
        crop_img = r_img[0:img_size, 0:img_size]

    return crop_img

def kMeans(img_rgb, number_of_clusters, attempts):
    # KMeans Processing
    img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
    K = number_of_clusters
    onion_2D = img_hsv.reshape((-1,3)) #Reshape to 2 dimensional image (65536, 3)
    onion_2D = np.float32(onion_2D)

    CONSTANT_1 = 10
    CONSTANT_2 = 1.0
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, CONSTANT_1, CONSTANT_2)

    ret, label, center=cv2.kmeans(onion_2D, K, None, criteria, attempts, cv2.KMEANS_PP_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    result_image = res.reshape((img_rgb.shape))

    return result_image, label, center

def min_area_rect(img_rgb, original_img, thresh_low, thresh_high):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    
    # Use Otsu's method to calculate a dynamic threshold
    #a, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    #print(thresh.min(), thresh.max())
    #thresh_low = 0.5 * thresh
    #thresh_high = 1.5 * thresh
    # Apply edge detection using the dynamic threshold
    edges = cv2.Canny(gray, thresh_low, thresh_high)
    
    # Find contours in the edged image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #print('Length Contours: ', len(contours))
    # Find the contour with the maximum area
    max_contour = max(contours, key=cv2.contourArea)
    
    # Get the rotated bounding box of the contour
    rect = cv2.minAreaRect(max_contour)
    box = cv2.boxPoints(rect)
    box = np.intp(box)
    
    # Draw the rotated bounding box on the original image
    #result_image = img_rgb.copy()
    #cv2.drawContours(result_image, [box], 0, (0, 255, 0), 2)

    result_image = original_img.copy()
    cv2.drawContours(result_image, [box], 0, (0, 255, 0), 2)
    
    return result_image

def get_object_color(center, cluster_img_hsv):
    for point in center:
        edge_point = 0
        if (point == cluster_img_hsv[0][0]).all():
            edge_point += 1

        if (point == cluster_img_hsv[0][255]).all():
            edge_point += 1

        if (point == cluster_img_hsv[255][255]).all():
            edge_point += 1

        if (point == cluster_img_hsv[255][0]).all():
            edge_point += 1

        if edge_point >= 3:
            #print(f"Color Background found: {point}")
            pass

        if edge_point < 3:
            #print(f"Color Object found: {point}")
            return point

In [4]:
path = "G:/Meine Ablage/KI_Projekt/Bilder"
#vegi = "kartoffel_"
folders = [folder for folder in os.listdir(path)]
print(folders)

['Zwiebel', 'Karotte', 'Kartoffel', 'Zwiebel_Trieb', 'Karotte_Trieb', 'Kartoffel_Trieb', 'Sonstige', 'BildermitBounding', 'desktop.ini']


In [40]:
def preprocess_images(image_folders):
    for fold in image_folders:
        print(f"Edit Images in: {fold}\n")
        # Get all images in Folder
        for i, fname in enumerate(os.listdir(f"G:/Meine Ablage/KI_Projekt/Bilder/{fold}")):
            if i == 150:
                # At the moment draw only boxes for the first 100 images
                break

            path = f"G:\Meine Ablage\KI_Projekt\Bilder\{fold}\{fname}"
            vegi = cv2.imread(path.replace("\\","/"))
            vegi = cv2.cvtColor(vegi, cv2.COLOR_BGR2RGB)
            #print("############ ORIGINAL ########")
            #plt.imshow(vegi)
            #plt.show()
            vegi_sq = resize_to_square(vegi)

            median_vegi_rgb = cv2.medianBlur(vegi_sq, 5)
            
            clusterd_img_hsv, label, center = kMeans(median_vegi_rgb, number_of_clusters=2, attempts=50)
            #print("############ CLUSTER IMAGE HSV ########")
            #plt.imshow(clusterd_img_hsv)
            #plt.show()
            # Find correct cluster
            center_object = get_object_color(center, clusterd_img_hsv)
            #print("CENTER OBJECT: ", center_object)
            #print("EDGE: ", clusterd_img_hsv[0][0])
            #print("EDGE: ", clusterd_img_hsv[0][255])
            #print("EDGE: ", clusterd_img_hsv[255][0])
            #print("EDGE: ", clusterd_img_hsv[255][255])

            masked_img_gray, masked_img_rgb = mask_image(clusterd_img_hsv, median_vegi_rgb, border_low=center_object, border_high=center_object) #borders for hsv colorspace
            #print("############ MASKED IMG RGB ########")
            #plt.imshow(masked_img_rgb)
            #plt.show()
            result_box = min_area_rect(masked_img_rgb, vegi_sq, 0, 255)
            
            # Draw Box
            #vegi_copy = vegi.copy()
            #cv2.drawContours(vegi_copy, [box], 0, (0, 255, 0), 2)
            #print("############ RBG WITH BOUNDING BOX ########")
            #plt.imshow(result_box)
            #plt.show()
            # store box_img

            cv2.imwrite(f"G:\Meine Ablage\KI_Projekt\Bilder\BildermitBounding\{fold}_Bounding_Leon\{fname}", cv2.cvtColor(result_box, cv2.COLOR_BGR2RGB))

In [41]:
folders = ['Karotte', 'Zwiebel', 'Kartoffel']
preprocess_images(folders)

Edit Images in: Karotte

Edit Images in: Zwiebel

Edit Images in: Kartoffel

